In [129]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [130]:
# Učitavanje dataset-a
data = {
    'Maticna planeta': ['Earth', 'Europa', 'Europa', 'Europa', 'Mars', 'Earth', 'Mars', 'Earth', 'Mars', 'Earth', 'Earth', 'Earth', 'Earth', 'Earth', 'Earth', 'Earth', 'Europa', 'Earth', 'Europa', 'Mars', 'Mars', 'Europa', 'Earth', 'Earth'],

    'Kabina': ['F/0/5', 'B/1/P', 'B/1/P', 'B/1/P', 'F/1/P', 'G/1/S', 'F/3/P', 'F/4/P', 'F/5/P', 'G/0/P', 'F/6/P', 'E/0/S', 'E/0/S', 'E/0/S', 'E/0/S', 'E/0/S', 'C/2/S', 'F/6/S', 'C/0/P', 'F/9/P', 'F/9/P', 'D/1/S', 'G/3/P', 'G/7/S'],

    'Destinacija': ['TRAPPIST-1e', '55 Cancri e', 'TRAPPIST-1e', '55 Cancri e', 'TRAPPIST-1e', 'TRAPPIST-1e', '55 Cancri e', '55 Cancri e', 'TRAPPIST-1e', 'TRAPPIST-1e', '55 Cancri e', 'TRAPPIST-1e', '55 Cancri e', 'TRAPPIST-1e', 'PSO J318.5-22', 'TRAPPIST-1e', 'TRAPPIST-1e', 'TRAPPIST-1e', '55 Cancri e', 'TRAPPIST-1e', 'TRAPPIST-1e', '55 Cancri e', 'PSO J318.5-22', 'PSO J318.5-22'],

    'Starost': [24, 14, 34, 45, 32, 48, 27, 24, 45, 0, 14, 1, 49, 10, 1, 7, 62, 15, 34, 47, 2, 28, 21, 36],

    'VIP': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],

    'Ime': ['Juanna Vines', 'Erraiam Flatic', 'Altmair Flatic', 'Wezena Flatic', 'Berens Barne', 'Rensey Baketton', 'Flats Eccle', 'Carry Hugh', 'Alus Upread',  'Lyde Brightt', 'Philda Brightt', 'Almary Bratmaez', 'Glendy Bratmaez', 'Brenley Jacostanley', 'Mael Brantuarez', 'Terta Mcfaddemnon', 'Penton Fullided', 'Karard Brookenson', 'Anoyoni Unconsary', 'Coobix Datie', 'Cinets Datie', 'Ziba Qingwhedly', 'Cassa Cooperlandez', 'Warry Ayalazquez'],

    'Da li je prevezen': [True, True, True, True, True, False, False, False, True, True, False, False, False, True, False, False, True, False, False, True, True, True, True, False]
}

df = pd.DataFrame(data)

In [131]:
# Predprocesiranje podataka (kodiranje kategoričkih varijabli ili normalizacija numeričkih varijabli)
label_encoder = LabelEncoder()
df['Maticna planeta'] = label_encoder.fit_transform(df['Maticna planeta']) #kodiranje 'Maticna' Planeta kolone
df['Kabina'] = label_encoder.fit_transform(df['Kabina']) #Kodiranje 'Kabina' kolone
df['Destinacija'] = label_encoder.fit_transform(df['Destinacija']) #Kodiranje 'Destinacija' kolone

In [132]:
X = df.drop(['Da li je prevezen', 'Ime'], axis=1)
y = df['Da li je prevezen']

# Podjela podataka na skupove za treniranje i testiranje
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

In [133]:
from sklearn.feature_selection import f_classif

def partition_node(X, y):
    best_pvalue = np.inf
    best_feature = None
    best_split_value = None

    for feature in X.columns:
        f_scores, p_values = f_classif(X[[feature]], y)
        pvalue = p_values[0]
        if pvalue < best_pvalue:
            best_pvalue = pvalue
            best_feature = feature
            best_split_value = X[feature].mean()

    if best_feature is None:
        # Leaf node
        return y.mode().iloc[0]

    left_indices = list(X[best_feature] <= best_split_value)
    right_indices = list(X[best_feature] > best_split_value)

    return best_feature, best_split_value, left_indices, right_indices

In [134]:
def build_tree(X, y):
    feature, split_value, left_indices, right_indices = partition_node(X, y)

    if feature is None:
        # Leaf node
        return y.mode().iloc[0]

    left_X = X.loc[left_indices]
    left_y = y.loc[left_indices]
    right_X = X.loc[right_indices]
    right_y = y.loc[right_indices]

    tree = {
        'feature': feature,
        'split_value': split_value,
        'left': build_tree(left_X, left_y),
        'right': build_tree(right_X, right_y)
    }

    return tree

In [135]:
# Izgradnja stabla odlučivanja na osnovu odabranih atributa
tree = build_tree(X_train, y_train)

C:\Users\eriss\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\eriss\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\eriss\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\eriss\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\eriss\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univ

TypeError: cannot unpack non-iterable numpy.bool_ object